## Pytorch Lightning - JSMP starter notebook
Pytorch Lightning instroduction guide: https://pytorch-lightning.readthedocs.io/en/latest/introduction_guide.html

<div style="text-align: justify"> I have created this notebook aiming at encouraging people to use Pytorch Lightning (PL) since I have found it very interesting. PL has helped me to write cleaner code as well as reuse up to the 99% of it. Moreover, PL is easily scalable which is of paramount importance nowadays. Hope you enjoy this notebook and up-vote it if so. Ideas on how to improve not only the approach but also the code are welcome!  </div>

In [ ]:
import os
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from numba import njit
import pytorch_lightning as pl
import torch.nn.functional as F
from sklearn.utils import indexable
from typing import Iterable, Tuple, List
from sklearn.metrics import roc_auc_score
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, _num_samples

In [ ]:
PATH        = '../input/jane-street-market-prediction/'
OUT_PATH    = './'
EXP_NAME    = 'EXP001'
SEED        = 42
FOLD        = 0
BS          = 2048
EPOCHS      = 5
LR          = 1e-3
WD          = 1e-5
LAB_SMOOTH  = 1e-2
N_WORKERS   = 4
TRESH       = .5

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
@njit
def fillna_npwhere_njit(array, values):
    # https://www.kaggle.com/c/jane-street-market-prediction/discussion/201302
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
    
class LinBnDrop(nn.Sequential):
    # https://github.com/fastai/fastai/blob/master/fastai/layers.py#L166
    "Module grouping `BatchNorm1d`, `Dropout` and `Linear` layers"
    def __init__(self, n_in, n_out, bn=True, do=0., act=None, lin_first=False):
        layers = [nn.BatchNorm1d(n_out if lin_first else n_in)] if bn else []
        if do != 0: layers.append(nn.Dropout(do))
        lin = [nn.Linear(n_in, n_out, bias=not bn)]
        if act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)
        
class MLP(nn.Module):
    def __init__(self, num_feats, num_classes, h_units=[160, 160, 160], dropout=[.2, .2, .2, .2]):
        super(MLP, self).__init__()
        self.num_feats   = num_feats
        self.num_classes = num_classes
        self.h_units     = h_units
        self.dropout     = dropout

        n_in  = [num_feats] + h_units
        n_out = h_units + [num_classes]
        acts  = [nn.ReLU()] * len(h_units) + [nn.Sigmoid()]
        self.model = nn.Sequential(
            *[LinBnDrop(n_in[i], n_out[i], bn=True, do=dropout[i], act=acts[i]) for i in range(len(n_in))]
        )

    def forward(self, x):
        x = self.model(x)
        return x
    
def utility_score_bincount(date, weight, resp, action):
    # https://www.kaggle.com/c/jane-street-market-prediction/discussion/201257
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight    = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
seed_everything(SEED)

df = pd.read_csv(f'{PATH}/train.csv')

# JS seems to have modified the trading model around day 85
# https://www.kaggle.com/c/jane-street-market-prediction/discussion/201930
df = df.loc[df.date > 85].reset_index(drop=True)

# The samples with weight==0 have the highest return variance. Hence, the samples are not drawn from an identical distribution.
# https://www.kaggle.com/c/jane-street-market-prediction/discussion/201085
df = df.loc[df.weight != 0].reset_index(drop=True)

# Feature Engineering
df['cross_41_42_43'] = df['feature_41'] + df['feature_42'] + df['feature_43']
df['cross_1_2']      = df['feature_1'] / (df['feature_2'] + 1e-5)

# Create action column per resp
df['action']   = (df['resp'  ] > 0).astype('int')
df['action_1'] = (df['resp_1'] > 0).astype('int')
df['action_2'] = (df['resp_2'] > 0).astype('int')
df['action_3'] = (df['resp_3'] > 0).astype('int')
df['action_4'] = (df['resp_4'] > 0).astype('int')

# https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna#Time-Series-Cross-Validation
spliter = PurgedGroupTimeSeriesSplit(n_splits=5, max_train_group_size=7, group_gap=2, max_test_group_size=3)
for fold, (train_idx, valid_idx) in enumerate(spliter.split(X=df.index.values, y=None, groups=df.date.values)):
    np.save(f'{OUT_PATH}fold{fold}_train.npy', train_idx)
    np.save(f'{OUT_PATH}fold{fold}_valid.npy', valid_idx)
    
train_df = df.iloc[np.load(f'{OUT_PATH}fold{FOLD}_train.npy')].copy()
valid_df = df.iloc[np.load(f'{OUT_PATH}fold{FOLD}_valid.npy')].copy()
    
# Compute feature means to fill nans with
features = pd.read_csv(f'{PATH}/features.csv').feature.values.tolist() + ['cross_41_42_43', 'cross_1_2']
f_mean   = df[features].mean().values
np.save(f'{OUT_PATH}/f_mean.npy', f_mean)

del df

features = pd.read_csv(f'{PATH}/features.csv').feature.values.tolist() + ['cross_41_42_43', 'cross_1_2']
targets  = ['action', 'action_1', 'action_2', 'action_3', 'action_4']
f_mean   = np.load(f'{OUT_PATH}/f_mean.npy')

In [ ]:
class JSMPDataset(Dataset):
    def __init__(self, df, feats, targets, f_mean, extra_feats=[]):
        self.df          = df
        self.feats       = feats
        self.targets     = targets
        self.f_mean      = f_mean
        self.extra_feats = extra_feats

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        x = self.df.iloc[idx][self.feats].to_numpy()
        x = fillna_npwhere_njit(x, self.f_mean)
        y = self.df.iloc[idx][self.targets].to_numpy()
        
        if self.extra_feats:
            xtra_feats = self.df.iloc[idx][self.extra_feats].to_numpy()
            return torch.Tensor(x), torch.Tensor(y), torch.Tensor(xtra_feats)
        
        return torch.Tensor(x), torch.Tensor(y)

In [ ]:
class JSMPDataModule(pl.LightningDataModule):
    def __init__(self, train_df, valid_df, feats, targets, f_mean, bs=64, num_workers=4, prefetch_factor=2):
        super().__init__()
        self.train_df        = train_df
        self.valid_df        = valid_df
        self.feats           = feats
        self.targets         = targets
        self.f_mean          = f_mean
        self.bs              = bs
        self.num_workers     = num_workers
        self.prefetch_factor = prefetch_factor

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        self.train_ds = JSMPDataset(self.train_df, self.feats, self.targets, self.f_mean, extra_feats=['date', 'weight', 'resp'])
        self.valid_ds = JSMPDataset(self.valid_df, self.feats, self.targets, self.f_mean, extra_feats=['date', 'weight', 'resp'])
        self.test_ds  = None

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.bs, shuffle=True, num_workers=self.num_workers, prefetch_factor=self.prefetch_factor)

    def val_dataloader(self):
        return DataLoader(self.valid_ds, batch_size=self.bs, shuffle=False, num_workers=self.num_workers, prefetch_factor=self.prefetch_factor)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.bs, shuffle=False, num_workers=self.num_workers, prefetch_factor=self.prefetch_factor)

In [ ]:
class JSMPModule(pl.LightningModule):
    def __init__(self, num_feats, n_classes, h_units=[126, 126, 126], dropout=[.2, .2, .2, .2], lr=1e-3, wd=1e-5, lab_smooth=1e-2):
        super().__init__()
        self.save_hyperparameters()
        self.num_feats  = num_feats
        self.n_classes  = n_classes
        self.h_units    = h_units
        self.dropout    = dropout
        self.lr         = lr
        self.wd         = wd
        self.lab_smooth = lab_smooth
        
        self.model     = MLP(num_feats, n_classes, h_units, dropout)
        self.loss      = SmoothBCEwLogits(smoothing=lab_smooth)
        
    def forward(self, features):
        return self.model(features)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.wd)
    
    def training_step(self, batch, batch_idx):
        x, y, extra_feats  = batch
        
        y_hat = self.model(x)
        loss  = self.loss(y_hat, y.type_as(y_hat))
        
        metrics = {'loss': loss, 'summary': {'train_loss': loss, 'y': y, 'y_hat': y_hat, 'date': extra_feats[:, 0], 'weight': extra_feats[:, 1], 'resp': extra_feats[:, 2]}}
            
        return metrics
    
    def training_epoch_end(self, outputs):
        y      = torch.cat([x['summary']['y'     ] for x in outputs])
        y_hat  = torch.cat([x['summary']['y_hat' ] for x in outputs])
        date   = torch.cat([x['summary']['date'  ] for x in outputs])
        weight = torch.cat([x['summary']['weight'] for x in outputs])
        resp   = torch.cat([x['summary']['resp'  ] for x in outputs])
        loss   = torch.stack([x['summary']['train_loss'] for x in outputs])
        
        loss  = loss.mean()            
        m_auc = roc_auc_score(y.detach().cpu().numpy(), y_hat.detach().cpu().numpy())
        
        y_hat_action = y_hat.mean(1)
        y_hat_action = (y_hat_action >= TRESH).long()
        
        m_utility = utility_score_bincount(date.long().detach().cpu().numpy(), weight.detach().cpu().numpy(),
                resp.detach().cpu().numpy(), y_hat_action.detach().cpu().numpy())
        
        print(f'Epoch {self.current_epoch} -- [train] -- loss:{loss:.4f} | auc:{m_auc:.4f} | utility:{m_utility:.4f}')

    def validation_step(self, batch, batch_idx):
        x, y, extra_feats  = batch
                        
        y_hat = self.model(x)
        loss  = self.loss(y_hat, y.type_as(y_hat))
                
        metrics = {'valid_loss': loss, 'y': y, 'y_hat': y_hat, 'date': extra_feats[:, 0], 'weight': extra_feats[:, 1], 'resp': extra_feats[:, 2]}
        
        return metrics
            
    def validation_epoch_end(self, metrics):
                
        y      = torch.cat([x['y'     ] for x in metrics])
        y_hat  = torch.cat([x['y_hat' ] for x in metrics])
        date   = torch.cat([x['date'  ] for x in metrics])
        weight = torch.cat([x['weight'] for x in metrics])
        resp   = torch.cat([x['resp'  ] for x in metrics])
        loss   = torch.stack([x['valid_loss'] for x in metrics])
                
        if y.flatten().float().mean() > 0: # skip sanity check
            
            loss  = loss.mean()            
            m_auc = roc_auc_score(y.detach().cpu().numpy(), y_hat.detach().cpu().numpy())
                
            y_hat_action = torch.median(y_hat, axis=1).values
            y_hat_action = (y_hat_action >= TRESH).long()
                                        
            m_utility = utility_score_bincount(date.long().detach().cpu().numpy(), weight.detach().cpu().numpy(),
                resp.detach().cpu().numpy(), y_hat_action.detach().cpu().numpy())
                
            print(f'Epoch {self.current_epoch} -- [valid] -- loss:{loss:.4f} | auc:{m_auc:.4f} | utility:{m_utility:.4f}')
            
            self.log('valid_auc'    , m_auc)
            self.log('valid_loss'   , loss)
            self.log('valid_utility', m_utility)
        

In [ ]:
checkpoint_callback     = pl.callbacks.ModelCheckpoint(filepath=f'{OUT_PATH}/{EXP_NAME}-fold{FOLD}', monitor='valid_auc', mode='max', save_weights_only=True, save_last=True, verbose=True)
early_stopping_callback = pl.callbacks.early_stopping.EarlyStopping(monitor='valid_auc', mode='max', min_delta=.001, patience=20, verbose=True)

dm      = JSMPDataModule(train_df, valid_df, features, targets, f_mean, bs=BS, num_workers=N_WORKERS)
model   = JSMPModule(num_feats=len(features), n_classes=len(targets), h_units=[126, 126, 126], dropout=[.2, .2, .2, .2], lr=LR, wd=WD, lab_smooth=LAB_SMOOTH)
trainer = pl.Trainer(gpus=[0], callbacks=[checkpoint_callback, early_stopping_callback], max_epochs=EPOCHS, accelerator='dp')
trainer.fit(model, dm)